## Torchxrayvision 
Torchxrayvision is a platform based on pretrained models from chest x ray radiographies. The purpose ofthis notebook is managing the torchxrayvision framewrl

In [1]:
%%capture 
!pip install torchxrayvision
from google.colab import drive
import os 
import torchxrayvision as xrv

In [2]:
drive.mount('/content/drive')
%cd drive/MyDrive
os.chdir('./TFG/torchxrayvision/')
base_path = os.getcwd()
base_path

Mounted at /content/drive
/content/drive/MyDrive


'/content/drive/MyDrive/TFG/torchxrayvision'

In [3]:
os.listdir(base_path+"/data")

['RALO-Dataset.zip',
 '.ipynb_checkpoints',
 'volumes.zip',
 'metadata.csv',
 'NLMCXR_png.zip']

In [ ]:
# from zipfile import ZipFile

# with ZipFile(f'{base_path}/data/NLMCXR_png.zip', 'r') as zipObj:
#    # Extract all the contents of zip file in current directory
#    zipObj.extractall('NLM')

### 1.Models 
The models are already pretrained with it corresponding classifiers. To access to models we have modules in torchxrayvision to load pretrained models in pytorch

In [ ]:
# Generic function to load any core model
model = xrv.models.get_model(weights="densenet121-res224-all")
# # DenseNet 224x224 model trained on multiple datasets
# model = xrv.models.DenseNet(weights="densenet121-res224-all")
# # DenseNet trained on just the RSNA Pneumonia dataset
# model = xrv.models.DenseNet(weights="densenet121-res224-rsna")
# # ResNet 512x512 model trained on multiple datasets
# model = xrv.models.ResNet(weights="resnet50-res512-all")


If this fails you can run `wget https://github.com/mlmed/torchxrayvision/releases/download/v1/nih-pc-chex-mimic_ch-google-openi-kaggle-densenet121-d121-tw-lr001-rot45-tr15-sc15-seed0-best.pt -O /root/.torchxrayvision/models_data/nih-pc-chex-mimic_ch-google-openi-kaggle-densenet121-d121-tw-lr001-rot45-tr15-sc15-seed0-best.pt`
[██████████████████████████████████████████████████]


#### 1.1 Baseline classifiers 
There are different model classifiers according to the purpose you want to predict

In [ ]:
# DenseNet121 from JF Healthcare for the CheXpert competition
model = xrv.baseline_models.jfhealthcare.DenseNet()
# # Official Stanford CheXpert model
# model = xrv.baseline_models.chexpert.DenseNet()


Exception: ignored

#### 1.2 Classifier interface
We have a method to predict the classifications of the output label

In [ ]:
# img = 'Choose an image to classify the labels'
# predictions = model(img)[0] # 0 is first element of batch
# dict(zip(model.pathologies,predictions.detach().numpy()))
# prediction = model(img)[:,model.pathologies.index("Consolidation")]


#### 1.3 Feature Extraction 
We also have a method to interpret the different features of the model and labels with UMAP visualizations

In [ ]:
# feats = model.features(img)


#### 1.4 Autoencoders 
We have a pretrained autoencoder network to generate synthethic data from certain distribution. We can encode the image onto a distribution and decode a distribution to get an chest x ray image


In [ ]:
# image = 'Image to select '
# ae = xrv.autoencoders.ResNetAE(weights="101-elastic")
# z = ae.encode(image)
# image2 = ae.decode(z)


## 3. Datasets 
The datsets if the torchzrayvision

In [4]:
'''
This is a dataset with Geographic Extend and Lung Opacity Features
'''
dataset = xrv.datasets.StonyBrookCOVID_Dataset(imgpath=base_path+'/StonyBrook/imgs',
csvpath=base_path+'/StonyBrook/data.csv')
'''
Covid19 Dataset
'''
dataset2 = xrv.datasets.COVID19_Dataset(imgpath=f'{base_path}/data/volumes.zip',csvpath=f'{base_path}/data/metadata.csv')

In [ ]:
os.path.join(base_path,'/data/NLM/NLMCXR_png.tgz')

'/data/NLM/NLMCXR_png.tgz'

In [5]:


dataset3 = xrv.datasets.Openi_Dataset( os.path.join(base_path,'/data/NLM/NLMCXR_png.tgz'),xmlpath=os.path.join(base_path,'NLMCXR_reports.tgz'))

### MedCam Visualization with images
Medcam is a library implemented in pytorch used to visualize medical images, maybe a trial will be useful to consider if the library works or not

In [4]:
%%capture
!pip install medcam

In [23]:
from torchvision import models, transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torch
import cv2

# Setup the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = xrv.models.get_model(weights="densenet121-res224-all")
model.to(device=device)
model.eval()

def load_image(image_path):
    raw_image = cv2.imread(image_path)
    raw_image = cv2.resize(raw_image, (224,) * 2)
    image = transforms.Compose(
        [
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]
    )(raw_image[..., ::-1].copy())
    image = image.to(device)
    return image

# Load the dataset

'''
You must labels the images according to each of the true target it has 

'''
dataset = ImageFolder(f'NLM2/', loader=load_image)
# Set up the dataloader
'''
Load the dataset
'''
data_loader = DataLoader(dataset, batch_size=1, shuffle=False)

FileNotFoundError: ignored

In [ ]:
'''
Called to the MedCam Library so that 
loads the visualization
In output dir the module outputs the image with gradcam, c2d or whatever backend you use
'''
from medcam import medcam

model = medcam.inject(model, output_dir='attention_maps', backend='gcam', layer='layer4', label='best', save_maps=True)

In [ ]:
# import os, sys, tarfile

# def extract(tar_url, extract_path='.'):
#     tar = tarfile.open(tar_url, 'r')
#     for item in tar:
#         tar.extract(item, extract_path)
#         if item.name.find(".tgz") != -1 or item.name.find(".tar") != -1:
#             extract(item.name, "./" + item.name[:item.name.rfind('/')],extract_path)


# extract(f'{base_path}/NLM/NLMCXR_png.tgz',extract_path='NLM2')

# except:
#     name = os.path.basename(sys.argv[0])
#     print name[:name.rfind('.')], '<filename>'
